# HLS & ERA5-Land Data Extraction at Water Quality Observations

> Description
- This code is for extracting HLS bands and ERA5 Land data for each basin  
  using QA band (quality flag for water) and Density-based spatial clustering of applications with noise (DBSCAN).

- This code is parallelized using **Joblib**.
  
- The dates are based on HLS data.  
  (In the case of South Korea, HLS observation times are around 2 AM UTC. Accordingly, ERA5-Land data for the corresponding 2 AM UTC timestamp was extracted.)

## 1. Basic Settings

### 1.1. Import Libraries

In [ ]:
# Import libraries
import os
import sys
import platform
import importlib
import pyproj
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import geopandas as gpd
from netCDF4 import Dataset
import rasterio
from rasterio.transform import rowcol
from scipy.spatial import cKDTree
from sklearn.cluster import DBSCAN
from joblib import Parallel, delayed

# Set base directory
if platform.system() == 'Darwin':  # macOS
    base_FP = '/Users/lsj'
    cpuserver_data_FP = base_FP + '/cpuserver_data'
    nas_data_FP = '/Volumes/qnap_nas'
elif platform.system() == 'Linux':  # Linux systems (Workstation / CPU Server GPU Server)
    base_FP = '/home/seongjun'
    cpuserver_data_FP = base_FP + '/cpuserver_data' # Workstation / GPU Server
    if not os.path.exists(cpuserver_data_FP):
        cpuserver_data_FP = '/data' # CPU Server
    nas_data_FP = base_FP + '/NAS'

# Add Python modules path
sys.path.append(os.path.join(base_FP, 'python_modules'))

# Private modules
import HydroAI.Data as Data
import remote_sensing.HLS as HLS
import Slack.slack_notifier as Slack
importlib.reload(Data);
importlib.reload(HLS);
importlib.reload(Slack);

# Slack
env_path = os.path.join(base_FP, '.env')
notipy = Slack.Notifier(env_path)

### 1.2. Define Product and paths

In [2]:
# Define product
product = 'S30'
band_list = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'qa']

# Define paths
obs_df_path = os.path.join(nas_data_FP, f'water_quality/WEIS_data/WEIS_obs_filtered_2013_2024.csv')
obs_geojson_path = os.path.join(nas_data_FP, f'water_quality/DBSCAN/obs_water_clusters.json')
shp_file_path = os.path.join(nas_data_FP, 'water_quality/shp_files/water_body_shp/water.shp')
hls_dir_path = os.path.join(nas_data_FP, f'HLS/Korea/{product}')
era5_dir_path = os.path.join(cpuserver_data_FP, 'ERA5_Land/Korea')
save_dir_path = os.path.join(nas_data_FP, f'water_quality/final_dataset/{product}')
os.makedirs(save_dir_path, exist_ok=True)

# Get date list
date_list = sorted([dir for dir in os.listdir(hls_dir_path) if dir.isdigit() and len(dir) == 8])

print(f"Number of {product} dates: {len(date_list)}")

Number of S30 dates: 1115


### 1.3. Hyperparameters for DBSCAN

In [3]:
# DBSCAN Hyperparameter
eps = 0.00045
EPS_RAD = np.radians(eps)
MIN_SAMPLES = 5
ALGORITHM = 'ball_tree'
METRIC = 'haversine'

# General parameters
DISTANCE_THRESHOLD = 300 # meters

## 2. Load Datasets 

### 2.1. In-Situ Data (Water Qaulity Observations)

In [4]:
obs_df = pd.read_csv(obs_df_path)
required_columns = ['latitude', 'longitude', 'wmcymd']
if all(col in obs_df.columns for col in required_columns):
    obs_df.rename(columns={'latitude': 'obs_lat', 'longitude': 'obs_lon', 'wmcymd': 'obs_date'}, inplace=True)
obs_df

,ptNo,ptNm,obs_lat,obs_lon,obs_date,wmdep,itemTemp,itemCloa,itemTn,itemTp,itemDoc
0,1001A05,송천1,37.658611,128.676389,3/21/13,NaN,2.5,1.5,3.785,0.020,10.5
1,1001A05,송천1,37.658611,128.676389,4/4/13,NaN,10.7,2.8,3.342,0.023,12.6
2,1001A05,송천1,37.658611,128.676389,5/29/13,NaN,17.8,5.5,2.748,0.026,10.2
3,1001A05,송천1,37.658611,128.676389,6/25/13,NaN,23.4,10.3,5.049,0.135,9.4
4,1001A05,송천1,37.658611,128.676389,7/4/13,NaN,20.6,3.9,4.419,0.045,9.7
...,...,...,...,...,...,...,...,...,...,...,...
244879,5303D70,수락저수지,34.846111,126.265000,10/23/23,NaN,19.2,16.1,0.695,0.029,7.5
244880,5303D70,수락저수지,34.846111,126.265000,6/18/24,NaN,26.2,4.4,0.304,0.010,9.5
244881,5303D70,수락저수지,34.846111,126.265000,8/6/24,NaN,31.7,10.5,0.558,0.014,10.1
244882,5303D70,수락저수지,34.846111,126.265000,9/6/24,NaN,30.0,6.8,0.270,0.014,8.8


## 3. Extract HLS and ERA5 Land data for each date

> Warning !
* This code assumes that all HLS bands have the **same shape**.
* If the HLS bands have different shapes, you need to adjust the code accordingly.
* Please check the shape of the HLS bands before running the code.

### 3.1. Define function

In [5]:
def process_single_date(date, obs_df, hls_dir_path, era5_dir_path, save_dir_path, product, band_list):
    """
    Processes all data for a single date and saves the result to a file.
    """
    try:
        # Checkpoint: skip if the file already exists
        daily_output_path = os.path.join(save_dir_path, f'{product}_{date}.feather')
        if os.path.exists(daily_output_path):
            return f"Skipping {date} because it already exists"

        # =============================================================================#
        # HLS pre-processing
        # =============================================================================#

        # 1. Temporal filtering
        # 1.1. Integrate date type
        obs_df_filtered = obs_df.copy()
        obs_df_filtered['obs_date'] = pd.to_datetime(obs_df_filtered['obs_date'], format='%m/%d/%y')
        hls_pd_date = pd.to_datetime(date, format='%Y%m%d')

        # 1.2. Temporal filtering (1 day window)
        time_window = timedelta(days=1)
        date_mask = (obs_df_filtered['obs_date'] >= hls_pd_date - time_window) & \
                    (obs_df_filtered['obs_date'] <= hls_pd_date + time_window)

        obs_df_filtered = obs_df_filtered[date_mask].copy()

        ##----------------------------------------------------------------------------##

        # 2. Load HLS data
        # 2.1 Read HLS data
        hls_path = os.path.join(hls_dir_path, date)
        HLS_reader = HLS.BandReader(hls_path, product, date)
        _, transform, _ = HLS_reader.get_band_with_transform(band_list[0], product)  # Get for the first band (load transform)

        # 2.2. Read entire HLS bands
        NO_DATA_VALUE = -0.9999 # (-9999 * scale factor)
        band_data = {}
        for band in band_list:
            band_data[band] = getattr(HLS_reader, band)
            band_data[band][band_data[band] == NO_DATA_VALUE] = np.nan

        ##----------------------------------------------------------------------------##

        # 3. Masking noisy pixels
        # 3.1. Create a noise mask from QA band
        # QA band bit number:
        # 0 = Cirrus
        # 1 = Cloud
        # 2 = Adjacent to cloud/shadow
        # 3 = Cloud shadow
        # 4 = Snow/ice
        # 5 = Water
        # 6-7 = aerosol optical thickness (AOT) level

        # 96 (01100000) = Water, low confidence
        # 160 (10100000) = Water, medium confidence
        # 224 (11100000) = Water, high confidence
        # We create a 'noise_mask' that is True for everything EXCEPT water.

        water_mask = np.isin(band_data['qa'], [96, 160, 224])
        noise_mask = ~np.isin(band_data['qa'], [96, 160, 224])

        # 3.2. Apply the mask to all bands
        # This sets the noisy pixels in all bands to NaN (Not a Number)
        for band in band_list:
            if band == 'qa': # Skip QA band (integer)
                continue
            band_data[band] = band_data[band].astype(np.float32) # Convert to float to allow NaN
            band_data[band][noise_mask] = np.nan

        # 3.3. Clip negative values to 0 for all bands
        # This ensures all pixel values are non-negative
        for band in band_list:
            if band != 'qa':
                band_data[band] = np.clip(band_data[band], 0, None)

        ##----------------------------------------------------------------------------##

        # 4. Calculate MNDWI for each observation point
        # 4.1. Get all observation coordinates from the DataFrame
        lons = obs_df_filtered['obs_lon'].values
        lats = obs_df_filtered['obs_lat'].values

        # 4.2. Convert all geographic coordinates to pixel coordinates (row, col) at once
        rows, cols = rowcol(transform, lons, lats)
        rows, cols = np.array(rows), np.array(cols)

        # 4.3. Create a mask to filter out points that are outside the image boundaries
        max_row, max_col = band_data['green'].shape
        valid_coords_mask = (rows >= 0) & (rows < max_row) & (cols >= 0) & (cols < max_col)

        # 4.4. Filter out points that are outside the image boundaries
        is_water_at_point = np.zeros(len(obs_df_filtered), dtype=bool)
        valid_rows = rows[valid_coords_mask]
        valid_cols = cols[valid_coords_mask]

        is_water_values = ~np.isnan(band_data['green'][valid_rows, valid_cols])
        is_water_at_point[valid_coords_mask] = is_water_values

        obs_noise_filtered_df = obs_df_filtered[is_water_at_point].reset_index(drop=True)

        ##----------------------------------------------------------------------------##

        # 5. DBSCAN Clustering
        # 5.1. Get the pixel coordinates (row, col) of all water pixels
        water_pixel_indices = np.argwhere(water_mask)

        if water_pixel_indices.size == 0:
            water_labels = np.array([])
            water_gdf = gpd.GeoDataFrame({'cluster_label': [], 'geometry': []}, crs="EPSG:4326")
        else:
            rows = water_pixel_indices[:, 0]
            cols = water_pixel_indices[:, 1]

            # 5.1.1. Convert pixel coordinates to geographic coordinates (lon, lat)
            water_lons, water_lats = rasterio.transform.xy(transform, rows, cols)

            # 5.1.2. Convert to radians for Haversine distance
            coords_rad = np.vstack((np.radians(water_lats), np.radians(water_lons))).T

            # 5.1.3. Perform DBSCAN clustering
            # Note: You might need to adjust EPS and MIN_SAMPLES for this new approach
            db = DBSCAN(eps=EPS_RAD,
                        min_samples=MIN_SAMPLES,
                        algorithm=ALGORITHM,
                        metric=METRIC,
                        n_jobs=-1
                        ).fit(coords_rad)
            water_labels = db.labels_

            # 5.1.4. Create a GeoDataFrame for all clustered water pixels
            water_gdf = gpd.GeoDataFrame({
                'cluster_label': water_labels,
                'geometry': gpd.points_from_xy(water_lons, water_lats)
            }, crs="EPSG:4326")

            water_gdf = water_gdf[water_gdf['cluster_label'] != -1]

        ##----------------------------------------------------------------------------##

        # 6. GeoDataFrame for Obs & HLS
        # 6.1. Create a GeoDataFrame for the observation stations from obs_df_filtered
        stations_gdf = gpd.GeoDataFrame(
            obs_noise_filtered_df,
            geometry=gpd.points_from_xy(obs_noise_filtered_df['obs_lon'], obs_noise_filtered_df['obs_lat']),
            crs="EPSG:4326"
        )

        # 6.2. Define the projected CRS for Korea (in meters)
        PROJECTED_CRS = "EPSG:5186" # UTM-K

        # 6.3. Re-project both GeoDataFrames to the projected CRS
        stations_gdf_proj = stations_gdf.to_crs(PROJECTED_CRS)
        water_gdf_proj = water_gdf.to_crs(PROJECTED_CRS)

        # 6.4. Use sjoin_nearest on the re-projected data
        # Now, 'max_distance' can be set directly in meters.
        final_matchup_gdf_proj = gpd.sjoin_nearest(
            stations_gdf_proj,
            water_gdf_proj,
            how='left',
            max_distance=DISTANCE_THRESHOLD # Use the meter-based threshold directly
        )

        # 6.5. Convert the result back to the original CRS (WGS84) for consistency
        final_matchup_gdf = final_matchup_gdf_proj.to_crs(stations_gdf.crs)

        # 6.6. Insert HLS date
        final_matchup_gdf['hls_date'] = pd.to_datetime(date, format='%Y%m%d')
        final_matchup_gdf.insert(5, 'hls_date', final_matchup_gdf.pop('hls_date'))

        # 6.7. Drop the extra index column created by the join
        final_matchup_gdf = final_matchup_gdf.drop(columns=['index_right'])

        ##----------------------------------------------------------------------------##

        # 7. Extract band values for all water pixels
        # 7.1. Convert longitude/latitude to pixel coordinates (row, col)
        water_lons = water_gdf.geometry.x
        water_lats = water_gdf.geometry.y
        rows, cols = rasterio.transform.rowcol(transform, water_lons, water_lats)

        # 7.2. Add pixel values of each band to water_gdf as new columns
        for band in band_list:
            if band != 'qa':
                water_gdf[band] = band_data[band][rows, cols]

        # 7.3. Calculate statistics for each cluster
        # 7.3.1. Define the statistics to calculate
        # Example: {'green': ['mean', 'median', 'std'], 'red': ['mean', 'median', 'std'], ...}
        aggregations = {band: ['mean', 'median', 'std'] for band in band_list if band != 'qa'}

        # 7.3.2. Calculate statistics for each cluster
        cluster_stats = water_gdf.groupby('cluster_label').agg(aggregations)

        # 7.3.3. Change the column names of the calculated statistics
        # Example: ('green', 'median') -> 'green', ('green', 'std') -> 'green_std'
        cluster_stats.columns = ['_'.join(col).strip() for col in cluster_stats.columns.values]

        ##----------------------------------------------------------------------------##

        # 8. Merge statistics into final_matchup_gdf
        # 8.1. Merge on 'cluster_label'
        obs_hls_df = pd.merge(
            final_matchup_gdf,
            cluster_stats,
            on='cluster_label',
            how='left' # Keep all rows from final_matchup_gdf
        )

        # 8.2. Add 'sensor' column at the 5th position (index 5)
        obs_hls_df.insert(5, 'sensor', product)
        final_obs_hls_df = obs_hls_df.copy()

        ##----------------------------------------------------------------------------##

        # 9. Calculate satellite-based indices
        with np.errstate(divide='ignore', invalid='ignore'):
            # 1) NDWI
            final_obs_hls_df['NDWI'] = (final_obs_hls_df['nir_mean'] - final_obs_hls_df['swir1_mean']) / (final_obs_hls_df['nir_mean'] + final_obs_hls_df['swir1_mean'])
            # 2) MNDWI
            final_obs_hls_df['MNDWI'] = (final_obs_hls_df['green_mean'] - final_obs_hls_df['swir1_mean']) / (final_obs_hls_df['green_mean'] + final_obs_hls_df['swir1_mean'])
            # 3) NDVI (for water)
            final_obs_hls_df['NDVI'] = (final_obs_hls_df['nir_mean'] - final_obs_hls_df['red_mean']) / (final_obs_hls_df['nir_mean'] + final_obs_hls_df['red_mean'])
            # 4) Blue-Green Ratio
            final_obs_hls_df['Green_Blue_Ratio'] = final_obs_hls_df['green_mean'] / final_obs_hls_df['blue_mean']
            # 5) NIR-Red Ratio
            final_obs_hls_df['NIR_Red_Ratio'] = final_obs_hls_df['nir_mean'] / final_obs_hls_df['red_mean']

        ##----------------------------------------------------------------------------##

        # 10. Replace inf and -inf with NaN
        final_obs_hls_df.replace([np.inf, -np.inf], np.nan, inplace=True)

        # =============================================================================#
        # ERA5 pre-processing
        # =============================================================================#

        # 1. Load ERA5 grid coordinates and data
        year = date[:4]
        era5_path = os.path.join(era5_dir_path, f'{year}/ERA5_land_{date}.nc')
        ds = Dataset(era5_path, 'r')

        ##----------------------------------------------------------------------------##

        # 2. Pre-process ERA5 grid coordinates
        # 2.1. Convert WGS84 (EPSG:4326) to UTM-K (EPSG:5186)
        transformer = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:5186", always_xy=True)

        lon_era5 = ds.variables['longitude'][:]
        lat_era5 = ds.variables['latitude'][:]
        lon_2d, lat_2d = np.meshgrid(lon_era5, lat_era5)

        # 2.2. Convert ERA5 grid coordinates to UTM-K (EPSG:5186)
        era5_x, era5_y = transformer.transform(lon_2d.ravel(), lat_2d.ravel())
        grid_points_proj = np.vstack([era5_x, era5_y]).T
        era5_tree = cKDTree(grid_points_proj)

        # 2.3. Convert observation coordinates to UTM-K (EPSG:5186)
        station_lon = final_obs_hls_df['obs_lon'].values
        station_lat = final_obs_hls_df['obs_lat'].values
        station_x, station_y = transformer.transform(station_lon, station_lat)
        station_points_proj = np.vstack([station_x, station_y]).T

        # 2.4. Find the index of the closest ERA5 grid point for each observation
        _, indices = era5_tree.query(station_points_proj, k=1)
        lat_indices, lon_indices = np.unravel_index(indices, lon_2d.shape)

        # 3. Pre-process precipitation data (tp)
        # 3.1. Calculate the date for the day before the target date
        target_date_dt = datetime.strptime(date, '%Y%m%d')
        era5_path_1 = os.path.join(era5_dir_path, f'{year}/ERA5_land_{(target_date_dt - timedelta(days=1)).strftime("%Y%m%d")}.nc')
        era5_path_2 = os.path.join(era5_dir_path, f'{year}/ERA5_land_{(target_date_dt - timedelta(days=2)).strftime("%Y%m%d")}.nc')

        # 3.2. Load the data for the previous days
        with Dataset(era5_path_1, 'r') as ds1:
            tp_day_1 = ds1.variables['tp'][:]
        with Dataset(era5_path_2, 'r') as ds2:
            tp_day_2 = ds2.variables['tp'][:]

        tp_day_0 = ds.variables['tp'][:]

        # 3.3. Calculate the cumulative precipitation and add it to final_obs_hls_df
        tp_24hr = np.sum(tp_day_0[:, lat_indices, lon_indices], axis=0)
        tp_48hr = tp_24hr + np.sum(tp_day_1[:, lat_indices, lon_indices], axis=0)
        tp_72hr = tp_48hr + np.sum(tp_day_2[:, lat_indices, lon_indices], axis=0)

        final_obs_hls_df['tp_24h'] = tp_24hr
        final_obs_hls_df['tp_48h'] = tp_48hr
        final_obs_hls_df['tp_72h'] = tp_72hr

        # 4. Pre-process other variables
        exclude_vars = {"number", "valid_time", "expver", "latitude", "longitude",
                        "stl2", "stl3", "stl4", "asn", "snowc", "rsn", "lict",
                        "sde", "sd", "swvl2", "swvl3", "swvl4", "ssrd", "strd", 'lai_hv', 'lai_lv'}

        for var_name in ds.variables:
            if var_name not in exclude_vars and var_name in ds.variables:
                # Calculate the daily mean value
                var_data = ds.variables[var_name][:]
                values = var_data[2, lat_indices, lon_indices]
                final_obs_hls_df[var_name] = values

        ds.close()

        # 5. Save the result
        if not final_obs_hls_df.empty:
            final_obs_hls_df.to_feather(daily_output_path)

    except Exception as e:
        return f"Failed to process {date}: {e}"

### 3.2. Parallel Processing for Extracting Data

In [6]:
# =================================================================================
# Parallel processing for HLS and ERA5 Land data
# =================================================================================
try:
    if __name__ == '__main__':
        # --- 1. Global variables and path settings ---
        # obs_df, date_list, hls_dir_path, era5_dir_path 등...

        # --- 2. Parallel execution ---
        # n_jobs=-1 means using all CPU cores. Adjust as needed to avoid server overload (e.g., 180)
        print(f"Starting parallel processing for {len(date_list)} dates...")

        results = Parallel(n_jobs=32, verbose=10)(
            delayed(process_single_date)(
                date, obs_df, hls_dir_path, era5_dir_path, save_dir_path, product, band_list
            ) for date in date_list
        )
        notipy.send_success()

except Exception as e:
    notipy.send_error(e)
    print(e)

Starting parallel processing for 1115 dates...


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.


[Parallel(n_jobs=32)]: Done   8 tasks      | elapsed:  1.7min
/home/seongjun/miniconda3/envs/lsj/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=32)]: Done  21 tasks      | elapsed:  4.0min
[Parallel(n_jobs=32)]: Done  34 tasks      | elapsed:  8.1min
[Parallel(n_jobs=32)]: Done  49 tasks      | elapsed: 10.6min
[Parallel(n_jobs=32)]: Done  64 tasks      | elapsed: 13.1min
[Parallel(n_jobs=32)]: Done  81 tasks      | elapsed: 15.4min
[Parallel(n_jobs=32)]: Done  98 tasks      | elapsed: 17.6min
[Parallel(n_jobs=32)]: Done 117 tasks      | elapsed: 22.0min
[Parallel(n_jobs=32)]: Done 136 tasks      | elapsed: 25.1min
[Parallel(n_jobs=32)]: Done 157 tasks      | elapsed: 32.1min
[Parallel(n_jobs=32)]: Done 178 tasks      | elapsed: 37.2min
[Parallel(n_jobs=32)]: Done 201 tasks      | e

### 3.3. Merge `.feather` to `.csv`

In [ ]:
product = 'S30'
save_dir_path = f'/home/seongjun/NAS/water_quality/final_dataset/{product}'

In [10]:
print("Consolidating all daily result files...")
file_list = [os.path.join(save_dir_path, f) for f in os.listdir(save_dir_path) if f.endswith('.feather')]
if file_list:
    all_dates_df = pd.concat([pd.read_feather(f) for f in file_list], ignore_index=True)
    all_dates_df = all_dates_df.drop(columns=['geometry', 'cluster_label'])
    all_dates_df = all_dates_df.sort_values(by=['ptNo', 'obs_date'])
    file_path = os.path.join(save_dir_path, f'{product}_obs_era5_land.csv')
    all_dates_df.to_csv(file_path, index=False, encoding='utf-8-sig')
    print(f"\nAll {len(file_list)} files have been consolidated into '{file_path}'")
else:
    print("No result files found to consolidate.")

Consolidating all daily result files...

All 648 files have been consolidated into '/home/seongjun/NAS/water_quality/final_dataset/L30/L30_obs_era5_land.csv'


### Results

In [11]:
# Read final dataset
final_dataset = pd.read_csv(os.path.join(save_dir_path, f'{product}_obs_era5_land.csv'))
final_dataset

,ptNo,ptNm,obs_lat,obs_lon,obs_date,sensor,hls_date,wmdep,itemTemp,itemCloa,...,pev,ro,es,ssro,sro,e,u10,v10,sp,tp
0,1001A30,조양강,37.439167,128.652778,2013-09-03,L30,2013-09-02,NaN,22.5,6.5,...,-0.001171,0.000078,4.365575e-11,0.000078,1.564622e-07,-0.000723,0.765411,-0.643417,94457.56,0.000020
1,1001A30,조양강,37.439167,128.652778,2014-10-15,L30,2014-10-14,NaN,13.0,1.7,...,-0.000919,0.000067,0.000000e+00,0.000067,0.000000e+00,-0.000337,-1.446213,-1.602402,94594.50,0.000000
2,1001A30,조양강,37.439167,128.652778,2015-07-07,L30,2015-07-06,NaN,22.1,8.2,...,-0.001295,0.000022,1.455191e-11,0.000022,5.960465e-08,-0.000769,-0.911423,-0.578705,94273.06,0.000007
3,1001A30,조양강,37.439167,128.652778,2015-09-09,L30,2015-09-08,NaN,18.7,2.5,...,-0.001488,0.000011,4.365575e-11,0.000011,0.000000e+00,-0.000699,-2.955643,-2.237575,94735.00,0.000000
4,1001A30,조양강,37.439167,128.652778,2016-12-07,L30,2016-12-06,NaN,2.9,0.5,...,-0.000516,0.000041,-4.924717e-06,0.000041,5.215406e-08,-0.000059,1.942169,0.243530,95017.31,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15477,5303D60,원산저수지,34.790833,126.110556,2024-06-18,L30,2024-06-18,NaN,21.9,0.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15478,5303D70,수락저수지,34.846111,126.265000,2019-06-13,L30,2019-06-13,NaN,20.5,11.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15479,5303D70,수락저수지,34.846111,126.265000,2023-08-04,L30,2023-08-03,NaN,23.0,8.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15480,5303D70,수락저수지,34.846111,126.265000,2024-06-18,L30,2024-06-18,NaN,26.2,4.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
